In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
#preparing our data
#keras has some functions that will automatically process the data and generate a flow of batches from the directory.
#expands on our dataset the line below
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
img=cv.imread("../input/dataset/dataset/train/Prohibition_Sign/100.jpeg")
img=cv.cvtColor(img,cv.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
#creating more general images with keras
#how many degrees randomly rotated (by 30 degrees indicated in the code)
#resizing: using the width and height shift range
#rescale: normalizing by dividing on the max
#shaer cuts away part of the image. cropping 20 percent.
# filling the missing pixels by the neasrest value.
image_gen=ImageDataGenerator(rotation_range=30, width_shift_range=0.1, height_shift_range=0.1,
                         rescale=1/255, shear_range=0.2,zoom_range=0.2,
                             vertical_flip=True, fill_mode="nearest", 
                            )

In [ ]:
#every time you run it you get a slightly different picture
plt.imshow(image_gen.random_transform(img))

In [ ]:
#gives information about your data set
image_gen.flow_from_directory("../input/dataset/dataset/train")

In [ ]:
# in this block and the block below I aim to find the average shape of the images in my dataset
dire1="../input/dataset/dataset/train/Prohibition_Sign"
import os
p1=[]
y_initial1=0
x_initial1=0
for i in os.listdir(dire1):
    p1.append(i)
for i in p1:
    path=os.path.join(dire1,i)
    img=cv.imread(path)
    y,x,c=img.shape
    y_initial1+=y
    x_initial1+=x

In [ ]:
dire2="../input/dataset/dataset/train/Other_Signs"
import os
p2=[]
y_initial2=0
x_initial2=0
for i in os.listdir(dire2):
    p2.append(i)
for i in p2:
    path=os.path.join(dire2,i)
    img=cv.imread(path)
    y,x,c=img.shape
    y_initial2+=y
    x_initial2+=x

In [ ]:
y_average=(y_initial1+y_initial2)//(len(p1)+len(p2))
x_average=(x_initial1+x_initial2)//(len(p1)+len(p2))
print ("average shape of the images in the dataset is :", (y_average, x_average))

In [ ]:
#creating your model
from keras.models import Sequential

In [ ]:
from keras.layers import Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Dense

In [ ]:
model = Sequential()

#adding layers to our neural networks
model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(600,600,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

#adding one more layer
model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=(600,600,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(120))
model.add(Activation("relu"))


#helps reduce overfitting by randomly turning off a percentage of the neurons
model.add(Dropout(0.5))


#the result is arbitrary: 0 for cat and 1 for dog.
model.add(Dense(1))
model.add(Activation("sigmoid"))

#calculating the cost function as the entropy formula
model.compile(loss="binary_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
#for train
batch_size=16
input_shape=(600,600,3)
train_image_gen= image_gen.flow_from_directory("../input/dataset/dataset/train",
                                        target_size=input_shape[:2],
                                              batch_size=batch_size,
                                              class_mode="binary")

In [ ]:
#for test
batch_size=16
input_shape=(600,600,3)
test_image_gen= image_gen.flow_from_directory("../input/dataset/dataset/test",
                                        target_size=input_shape[:2],
                                              batch_size=batch_size,
                                              class_mode="binary")

In [ ]:
#what index belongs to what class
train_image_gen.class_indices

In [ ]:
#epochs means doing a single full pass through the whole training set, so 
#5 epochs means going through 5 times.
results=model.fit_generator(train_image_gen,epochs=3,validation_data=test_image_gen)

In [ ]:
#evaluating the model
print (results.history["accuracy"])

#as we increase the number of epochs, our accuracy will increase because we have trained our
#data more.
plt.plot (results.history["accuracy"])

In [ ]:
#one of the reasons we got high accuracy was the the train data and test data were alot the same
#saving the model
model.save('project.h5')

In [ ]:
#predicting on new images
from keras.preprocessing import image

dire3="../input/dataset/dataset/test/Prohibition_Signs"

p3=[]

for i in os.listdir(dire3):
    p3.append(i)
for i in p3:
    path=os.path.join(dire3,i)
    img1=cv.imread(path)
    img1=cv.cvtColor(img1,cv.COLOR_BGR2RGB)
    img = image.load_img(path, target_size=(600, 600))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img/255
    prediction_prob = model.predict(img)
    if prediction_prob>=0.5:
        print (True)
    else:
        print ("Mistakenly detected as Not a prohibition sign")
        plt.imshow(img1)
        plt.show()
    print ("\n")
    print(f'Probability that image is a prohibition sign is: {prediction_prob} ')
    print ("----------------------------------------------")
    print ("\n")

In [ ]:
#predicting on new images
from keras.preprocessing import image

dire3="../input/dataset/dataset/test/Other_Signs"

p3=[]

for i in os.listdir(dire3):
    p3.append(i)
for i in p3:
    path=os.path.join(dire3,i)
    img1=cv.imread(path)
    img1=cv.cvtColor(img1,cv.COLOR_BGR2RGB)
    img = image.load_img(path, target_size=(600, 600))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img/255
    prediction_prob = model.predict(img)
    if prediction_prob>=0.5:
        print ("Mistakenly detected as a Prohibition sign")
        plt.imshow(img1)
        plt.show()
    else:
        print ("correctly detected as Not a prohibition sign")
    print ("\n")
    print(f'Probability that image is a prohibition sign is: {prediction_prob} ')
    print ("----------------------------------------------")
    print ("\n")

In [ ]:
#predicting on new images
from keras.preprocessing import image

path="../input/tireeee/54.jpeg"

img1=cv.imread(path)
img1=cv.cvtColor(img1,cv.COLOR_BGR2RGB)
img = image.load_img(path, target_size=(600, 600))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255
prediction_prob = model.predict(img)
if prediction_prob>=0.5:
    print ("Mistakenly detected as a Prohibition sign")
    plt.imshow(img1)
    plt.show()
else:
    print ("correctly detected as Not a prohibition sign")
print ("\n")
print(f'Probability that image is a prohibition sign is: {prediction_prob} ')
print ("----------------------------------------------")
print ("\n")
